In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # No limit on the width of the output

df_recipe = pd.read_csv("data_cleaned\\recipe_all_info.csv")
df_final_ingredients = pd.read_csv("data_cleaned\\final_ingredients.csv")
df_quantity = pd.read_csv("data_cleaned\\quantity_ingredient_final.csv")

In [3]:
# 'Iron, Fe_unit' #mg
# 'Vitamin A, RAE_unit' #ug
# 'Vitamin B-12_unit' #ug
# 'Energy_unit' #kcal and kj
# 'Cholesterol_unit' #mg

In [4]:
df_final_ingredients['Iron, Fe'] /= 1000
df_final_ingredients['Cholesterol'] /= 1000
df_final_ingredients['Vitamin A, RAE'] /= 1000000
df_final_ingredients['Vitamin B-12'] /= 1000000
df_final_ingredients.loc[df_final_ingredients['Energy_unit'] == 'kJ', 'Energy'] *= 0.239

df_final_ingredients[['Protein_unit', 'Fiber, total dietary_unit', 'Iron, Fe_unit', 'Vitamin A, RAE_unit', 'Vitamin B-12_unit',
                      'Fatty acids, total monounsaturated_unit', 'Cholesterol_unit']] = 'G'
df_final_ingredients['Energy_unit'] = 'KCAL'

In [5]:
merged_df = pd.merge(df_quantity, df_final_ingredients, on='ingredient_id')

multi_col = ['Protein', 'Fiber, total dietary', 'Iron, Fe', 'Vitamin A, RAE', 'Vitamin B-12', 'Energy',
             'Fatty acids, total monounsaturated', 'Cholesterol']

merged_df[multi_col] = merged_df[multi_col].multiply(merged_df['qty'] / 100, axis=0)

In [6]:
drop_col = ['ingredient_id', 'ingredient_name', 'category', 'unit', 'qty', 'foodCategory', 'name', 'vegCategory', 'oldCategory', 'Protein_unit',
            'Fiber, total dietary_unit', 'Iron, Fe_unit',  'Vitamin A, RAE_unit', 'Vitamin B-12_unit',  'Energy_unit',
            'Fatty acids, total monounsaturated_unit', 'Cholesterol_unit']

grouped_df = merged_df.groupby('recipe_id').sum().reset_index().drop(columns=drop_col)

In [7]:
df_recipe2 = df_recipe[['recipe_id', 'title', 'subtitle', 'ingredients']]
df_final = pd.merge(df_recipe2, grouped_df, on='recipe_id')
df_final

,recipe_id,title,subtitle,ingredients,Protein,"Fiber, total dietary","Iron, Fe","Vitamin A, RAE",Vitamin B-12,Energy,"Fatty acids, total monounsaturated",Cholesterol
0,214,Raspberry Chiffon Pie,NaN,"[924, 1414, 1613, 2196, 2222, 2797, 3334, 3334...",7.207800,0.384000,0.002095,0.000162,0.000000e+00,380.069040,3.657120,0.002400
1,215,Apricot Yogurt Parfaits,NaN,"[71, 76, 1286, 2019, 2196, 3567]",6.067950,2.824000,0.000541,0.000144,0.000000e+00,131.510000,0.119255,0.002400
2,216,Fresh Apricot Bavarian,NaN,"[1286, 2196, 3334, 3567, 3649]",4.899125,1.137500,0.000222,0.000055,0.000000e+00,141.273750,0.096688,0.002400
3,217,Fresh Peaches,with Banana Cream Whip,"[792, 1402, 2019]",1.092000,1.800000,0.000408,0.000029,0.000000e+00,70.200000,0.080400,0.000000
4,218,Canned Cherry Crisp,NaN,"[30, 34, 351, 374, 614, 770, 1684, 2948]",2.716000,2.433500,0.001968,0.000079,0.000000e+00,286.558000,2.162963,0.000600
...,...,...,...,...,...,...,...,...,...,...,...,...
838,1410,No-Bake Chocolate Cheesecake,NaN,"[35, 249, 374, 374, 496, 782, 792, 921, 1755, ...",8.951050,2.014583,0.003179,0.000257,6.806250e-08,659.687958,7.615858,0.077152
839,1411,Mexican Cocoa Torte,NaN,"[738, 1755, 1767, 2627, 3118, 3329, 3334]",5.639200,3.676000,0.005746,0.000197,7.680000e-08,543.064800,4.368000,0.061447
840,1426,-Pancakes-,NaN,"[30, 126, 2327, 3021, 3334, 3595, 3618]",7.517375,1.920000,0.002090,0.000001,2.437500e-09,288.768750,0.777875,0.007325
841,1427,-Waffles-,NaN,"[30, 125, 1696, 2327, 3021, 3334, 3618]",7.776750,1.920000,0.002088,0.000000,0.000000e+00,360.527812,2.295000,0.007875


In [8]:
df_final.to_csv("data_cleaned\\recipe_final.csv", index=False)